In [10]:
import csv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import Adam
from keras import backend as K
import glob
from PIL import Image
import json
import h5py
import os

In [26]:
json_string = open(os.path.join('./', 'cnn_functional_model.json')).read()
model = model_from_json(json_string)
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [12]:
weights_path = './fModel_weight.h5'
with h5py.File(weights_path) as f:
    layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]
    weight_value_tuples = []
    for k, name in enumerate(layer_names):
        if k >= len(model.layers):
            # 全結合層の重みは読み込まない
            break
        g = f[name]
        weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
        if len(weight_names):
            weight_values = [g[weight_name] for weight_name in weight_names]
            layer = model.layers[k]
            symbolic_weights = layer.trainable_weights + layer.non_trainable_weights
            if len(weight_values) != len(symbolic_weights):
                raise Exception('Layer #' + str(k) +
                            ' (named "' + layer.name +
                            '" in the current model) was found to '
                            'correspond to layer ' + name +
                            ' in the save file. '
                            'However the new layer ' + layer.name +
                            ' expects ' + str(len(symbolic_weights)) +
                            ' weights, but the saved weights have ' +
                            str(len(weight_values)) +
                            ' elements.')
            weight_value_tuples += zip(symbolic_weights, weight_values)

In [14]:
print('len layer names:', len(layer_names))
print('len model layers:', len(model.layers))
print('layer names:', layer_names)
print('model layer names:', [layer.name for layer in model.layers])

('len layer names:', 12)
('len model layers:', 12)
('layer names:', [u'input_8', u'convolution2d_53', u'convolution2d_54', u'convolution2d_55', u'dropout_13', u'flatten_13', u'dense_37', u'activation_37', u'dense_38', u'activation_38', u'dense_39', u'activation_39'])
('model layer names:', [u'input_1', u'convolution2d_32', u'convolution2d_33', u'convolution2d_34', u'dropout_6', u'flatten_6', u'dense_16', u'activation_16', u'dense_17', u'activation_17', u'dense_18', u'activation_18'])


In [32]:
from keras.utils.visualize_util import plot
plot(model, to_file="model.png", show_shapes=True)

In [33]:
get_3rd_layer_output = K.function([model.layers[0].input, K.learning_phase()],
                                  [model.layers[3].output])

# output in test mode = 0
layer_output = get_3rd_layer_output([X, 0])[0]

# output in train mode = 1
layer_output = get_3rd_layer_output([X, 1])[0]

NameError: name 'X' is not defined